# 🔋 통합 배터리 데이터 분석 파이프라인

**단일/연속 경로 & Toyo/PNE 자동 처리**

이 노트북은 모든 경로 타입에 대해 동일한 출력을 생성합니다:
1. ⚡ **수명 분석** (Cycle Analysis)
2. 📊 **프로파일 분석** (Profile Analysis)
3. 📈 **시각화** (Visualization)
4. 💾 **데이터베이스 저장** (Database Storage)

---

## 📂 지원 경로 타입

### 1. 연속 경로 (Toyo)
```python
paths = [
    r"Rawdata\250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc",
    r"Rawdata\250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc",
    r"Rawdata\250304_250404_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 201-300cyc",
    r"Rawdata\250317_251231_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 301-400cyc"
]
```

### 2. 연속 경로 (PNE)
```python
paths = [
    r"Rawdata\A1_MP1_4500mAh_T23_1",
    r"Rawdata\A1_MP1_4500mAh_T23_2",
    r"Rawdata\A1_MP1_4500mAh_T23_3"
]
```

### 3. 단일 경로 (Toyo)
```python
paths = [r"Rawdata\Q7M Sub ATL [45v 2068mAh] [23] - 250219r"]
```

### 4. 단일 경로 (PNE)
```python
paths = [r"Rawdata\A1_MP1_4500mAh_T23_3"]
```

---

## 🔧 환경 설정

In [ ]:
import os
import sys
from pathlib import Path

# 프로젝트 루트를 Python 경로에 추가
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ 프로젝트 루트: {project_root}")

In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 프로젝트 모듈
from src.core.cycler_detector import detect_cycler_type, validate_path_exists, get_channel_folders
from src.utils.config_models import ProfileConfig, CycleConfig, CyclerType
from src.database.session import init_db, session_scope
from src.database.repository import (
    TestProjectRepository, TestRunRepository,
    ProfileDataRepository, CycleDataRepository
)

print("✅ 모든 모듈 임포트 완료")

---

## 🎯 경로 설정

**아래 4가지 시나리오 중 하나를 선택하세요:**

In [ ]:
# ==============================
# 시나리오 선택 (주석 해제하여 사용)
# ==============================

# 1️⃣ 연속 경로 (Toyo)
paths = [
    r"Rawdata\250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc",
    r"Rawdata\250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc",
    r"Rawdata\250304_250404_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 201-300cyc",
    r"Rawdata\250317_251231_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 301-400cyc"
]
analysis_name = "ATL Q7M Inner 2C 상온수명 (전체)"

# 2️⃣ 연속 경로 (PNE)
# paths = [
#     r"Rawdata\A1_MP1_4500mAh_T23_1",
#     r"Rawdata\A1_MP1_4500mAh_T23_2",
#     r"Rawdata\A1_MP1_4500mAh_T23_3"
# ]
# analysis_name = "A1_MP1_4500mAh_T23 (전체)"

# 3️⃣ 단일 경로 (Toyo)
# paths = [r"Rawdata\Q7M Sub ATL [45v 2068mAh] [23] - 250219r"]
# analysis_name = "Q7M Sub ATL"

# 4️⃣ 단일 경로 (PNE)
# paths = [r"Rawdata\A1_MP1_4500mAh_T23_3"]
# analysis_name = "A1_MP1_4500mAh_T23_3"

# ==============================
# 공통 파라미터 설정
# ==============================
mincapacity = 0      # 최소 용량 (mAh)
firstCrate = 0.2     # 기준 C-rate
chkir = False        # DCIR 계산 여부

print(f"📂 분석 대상: {analysis_name}")
print(f"📁 경로 개수: {len(paths)}")
print(f"⚙️  파라미터: mincapacity={mincapacity}, firstCrate={firstCrate}, chkir={chkir}")

---

## 🔍 1단계: 경로 검증 및 Cycler 자동 감지

In [ ]:
# 경로 검증
print("🔍 경로 검증 중...\n")

valid_paths = []
cycler_types = []

for i, path in enumerate(paths, 1):
    try:
        # 경로 존재 확인
        validate_path_exists(path)
        
        # Cycler 타입 자동 감지
        cycler_type = detect_cycler_type(path)
        
        # 채널 폴더 확인
        channels = get_channel_folders(path, cycler_type)
        
        valid_paths.append(path)
        cycler_types.append(cycler_type)
        
        print(f"✅ [{i}/{len(paths)}] {os.path.basename(path)}")
        print(f"   └─ Cycler: {cycler_type.value}")
        print(f"   └─ 채널 수: {len(channels)}개\n")
        
    except Exception as e:
        print(f"❌ [{i}/{len(paths)}] {path}")
        print(f"   └─ 오류: {e}\n")

# 모든 경로가 같은 Cycler 타입인지 확인
if len(set(cycler_types)) > 1:
    print("⚠️  경고: 서로 다른 Cycler 타입이 혼합되어 있습니다!")
else:
    detected_cycler = cycler_types[0] if cycler_types else None
    print(f"✅ 감지된 Cycler 타입: {detected_cycler.value if detected_cycler else 'None'}")

print(f"\n📊 검증 완료: {len(valid_paths)}/{len(paths)} 경로 사용 가능")

---

## ⚡ 2단계: 프로파일 분석 (Profile Analysis)

Rate test 데이터 추출 및 기준 사이클 선정

In [ ]:
# Profile Loader 선택 (Cycler 타입에 따라)
if detected_cycler == CyclerType.TOYO:
    from src.core.toyo_loader import ToyoRateProfileLoader
    ProfileLoader = ToyoRateProfileLoader
elif detected_cycler == CyclerType.PNE:
    from src.core.pne_loader import PNERateProfileLoader
    ProfileLoader = PNERateProfileLoader
else:
    raise ValueError(f"지원하지 않는 Cycler 타입: {detected_cycler}")

print(f"✅ Profile Loader: {ProfileLoader.__name__}")

In [ ]:
# 첫 번째 경로로 프로파일 분석 수행
print(f"⚡ 프로파일 분석 시작...\n")

profile_config = ProfileConfig(
    raw_file_path=valid_paths[0],
    mincapacity=mincapacity,
    firstCrate=firstCrate
)

loader = ProfileLoader(profile_config)
profile_result = loader.load()

print(f"✅ 프로파일 분석 완료")
print(f"\n📊 분석 결과:")
print(f"   • 총 사이클 수: {profile_result.total_cycles}")
print(f"   • Rate 사이클 수: {profile_result.rate_cycles}")
print(f"   • 기준 사이클: {profile_result.base_cycle}")
print(f"   • 기준 C-rate: {profile_result.base_crate}")
print(f"   • 기준 용량: {profile_result.base_capacity:.2f} mAh")

# Rate test 데이터 표시
if profile_result.profile_data is not None and not profile_result.profile_data.empty:
    print(f"\n📈 Rate Test 데이터 ({len(profile_result.profile_data)}개 사이클):")
    display(profile_result.profile_data)
else:
    print("\n⚠️  Rate test 데이터가 없습니다.")

---

## 🔄 3단계: 수명 분석 (Cycle Analysis)

모든 경로의 수명 데이터 추출 및 통합

In [ ]:
# Cycle Analyzer 선택
if detected_cycler == CyclerType.TOYO:
    from src.core.toyo_cycle_analyzer import ToyoCycleAnalyzer
    CycleAnalyzer = ToyoCycleAnalyzer
elif detected_cycler == CyclerType.PNE:
    from src.core.pne_cycle_analyzer import PNECycleAnalyzer
    CycleAnalyzer = PNECycleAnalyzer

print(f"✅ Cycle Analyzer: {CycleAnalyzer.__name__}")

In [ ]:
# 모든 경로에 대해 수명 분석 수행
print(f"🔄 수명 분석 시작...\n")

all_cycle_data = []

for i, path in enumerate(valid_paths, 1):
    print(f"[{i}/{len(valid_paths)}] {os.path.basename(path)}")
    
    try:
        # Cycle 분석 설정
        cycle_config = CycleConfig(
            raw_file_path=path,
            mincapacity=mincapacity,
            firstCrate=firstCrate,
            chkir=chkir,
            base_cycle=profile_result.base_cycle,
            base_crate=profile_result.base_crate,
            base_capacity=profile_result.base_capacity
        )
        
        # 분석 실행
        analyzer = CycleAnalyzer(cycle_config)
        cycle_result = analyzer.analyze()
        
        if cycle_result.cycle_data is not None and not cycle_result.cycle_data.empty:
            all_cycle_data.append(cycle_result.cycle_data)
            print(f"   ✅ {len(cycle_result.cycle_data)} 사이클 추출\n")
        else:
            print(f"   ⚠️  데이터 없음\n")
            
    except Exception as e:
        print(f"   ❌ 오류: {e}\n")

# 모든 데이터 통합
if all_cycle_data:
    combined_cycle_data = pd.concat(all_cycle_data, ignore_index=True)
    print(f"✅ 수명 분석 완료")
    print(f"\n📊 통합 결과:")
    print(f"   • 총 사이클 수: {len(combined_cycle_data)}")
    print(f"   • 사이클 범위: {combined_cycle_data['Cycle'].min()} ~ {combined_cycle_data['Cycle'].max()}")
    print(f"   • 용량 범위: {combined_cycle_data['DChgCap'].min():.2f} ~ {combined_cycle_data['DChgCap'].max():.2f} mAh")
    
    # 데이터 샘플 표시
    print(f"\n📋 데이터 샘플 (처음 10개):")
    display(combined_cycle_data.head(10))
else:
    print("❌ 수명 데이터를 추출할 수 없습니다.")
    combined_cycle_data = pd.DataFrame()

---

## 📈 4단계: 시각화 (Visualization)

In [ ]:
if not combined_cycle_data.empty:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'📊 {analysis_name} - 수명 분석 결과', fontsize=16, fontweight='bold')
    
    # 1. 방전 용량 (Discharge Capacity)
    ax1 = axes[0, 0]
    ax1.plot(combined_cycle_data['Cycle'], combined_cycle_data['DChgCap'], 
             marker='o', markersize=3, linewidth=1.5, color='#2E86AB')
    ax1.set_xlabel('Cycle', fontsize=12)
    ax1.set_ylabel('방전 용량 (mAh)', fontsize=12)
    ax1.set_title('방전 용량 추이', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # 2. 충전 용량 (Charge Capacity)
    ax2 = axes[0, 1]
    ax2.plot(combined_cycle_data['Cycle'], combined_cycle_data['ChgCap'], 
             marker='o', markersize=3, linewidth=1.5, color='#A23B72')
    ax2.set_xlabel('Cycle', fontsize=12)
    ax2.set_ylabel('충전 용량 (mAh)', fontsize=12)
    ax2.set_title('충전 용량 추이', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # 3. 충방전 효율 (Efficiency)
    ax3 = axes[1, 0]
    efficiency = (combined_cycle_data['DChgCap'] / combined_cycle_data['ChgCap'] * 100).fillna(0)
    ax3.plot(combined_cycle_data['Cycle'], efficiency, 
             marker='o', markersize=3, linewidth=1.5, color='#F18F01')
    ax3.set_xlabel('Cycle', fontsize=12)
    ax3.set_ylabel('효율 (%)', fontsize=12)
    ax3.set_title('충방전 효율', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim([90, 105])
    
    # 4. 용량 유지율 (Capacity Retention)
    ax4 = axes[1, 1]
    if profile_result.base_capacity > 0:
        retention = (combined_cycle_data['DChgCap'] / profile_result.base_capacity * 100).fillna(0)
        ax4.plot(combined_cycle_data['Cycle'], retention, 
                 marker='o', markersize=3, linewidth=1.5, color='#6A994E')
        ax4.axhline(y=80, color='r', linestyle='--', linewidth=2, label='80% 기준선')
        ax4.set_xlabel('Cycle', fontsize=12)
        ax4.set_ylabel('용량 유지율 (%)', fontsize=12)
        ax4.set_title(f'용량 유지율 (기준: {profile_result.base_capacity:.2f} mAh)', 
                      fontsize=14, fontweight='bold')
        ax4.grid(True, alpha=0.3)
        ax4.legend()
    else:
        ax4.text(0.5, 0.5, '기준 용량 정보 없음', 
                ha='center', va='center', transform=ax4.transAxes, fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 시각화 완료")
else:
    print("⚠️  시각화할 데이터가 없습니다.")

---

## 💾 5단계: 데이터베이스 저장 (Database Storage)

In [ ]:
# 데이터베이스 초기화
db_path = "sqlite:///battery_analysis.db"
engine = init_db(db_path, echo=False)

print(f"✅ 데이터베이스 연결: {db_path}")

In [ ]:
# 프로젝트 및 테스트 런 생성
with session_scope() as session:
    # Repository 초기화
    project_repo = TestProjectRepository(session)
    run_repo = TestRunRepository(session)
    profile_repo = ProfileDataRepository(session)
    cycle_repo = CycleDataRepository(session)
    
    # 프로젝트 생성 또는 조회
    project = project_repo.get_by_name(analysis_name)
    if project is None:
        project = project_repo.create(
            name=analysis_name,
            description=f"자동 생성 - {detected_cycler.value} 분석"
        )
        print(f"✅ 새 프로젝트 생성: {project.name}")
    else:
        print(f"✅ 기존 프로젝트 사용: {project.name}")
    
    # 테스트 런 생성
    run_name = f"{analysis_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    test_run = run_repo.create(
        project_id=project.id,
        run_name=run_name,
        cycler_type=detected_cycler.value,
        base_capacity=profile_result.base_capacity,
        base_crate=profile_result.base_crate
    )
    print(f"✅ 테스트 런 생성: {test_run.run_name}")
    
    # 프로파일 데이터 저장
    if profile_result.profile_data is not None and not profile_result.profile_data.empty:
        profile_count = profile_repo.bulk_insert_from_dataframe(
            test_run_id=test_run.id,
            df=profile_result.profile_data
        )
        print(f"✅ 프로파일 데이터 저장: {profile_count}개")
    
    # 수명 데이터 저장
    if not combined_cycle_data.empty:
        cycle_count = cycle_repo.bulk_insert_from_dataframe(
            test_run_id=test_run.id,
            df=combined_cycle_data
        )
        print(f"✅ 수명 데이터 저장: {cycle_count}개")
    
    print(f"\n💾 데이터베이스 저장 완료!")
    print(f"   • 프로젝트 ID: {project.id}")
    print(f"   • 테스트 런 ID: {test_run.id}")

---

## 🔍 데이터베이스 조회 예제

In [ ]:
# 저장된 데이터 조회
with session_scope() as session:
    cycle_repo = CycleDataRepository(session)
    
    # 전체 사이클 데이터 조회
    df_from_db = cycle_repo.get_dataframe_by_run(test_run.id)
    
    print(f"📊 데이터베이스에서 조회한 데이터:")
    print(f"   • 총 레코드 수: {len(df_from_db)}")
    print(f"\n📋 데이터 샘플:")
    display(df_from_db.head(10))

---

## 📊 최종 요약

In [ ]:
print("="*80)
print(f"🎯 분석 완료: {analysis_name}")
print("="*80)
print(f"\n📂 경로 정보:")
print(f"   • 총 경로 수: {len(valid_paths)}")
print(f"   • Cycler 타입: {detected_cycler.value}")
for i, path in enumerate(valid_paths, 1):
    print(f"   [{i}] {os.path.basename(path)}")

print(f"\n📊 프로파일 분석:")
print(f"   • 총 사이클: {profile_result.total_cycles}")
print(f"   • Rate 사이클: {profile_result.rate_cycles}")
print(f"   • 기준 사이클: {profile_result.base_cycle}")
print(f"   • 기준 용량: {profile_result.base_capacity:.2f} mAh")

print(f"\n⚡ 수명 분석:")
if not combined_cycle_data.empty:
    print(f"   • 총 사이클: {len(combined_cycle_data)}")
    print(f"   • 사이클 범위: {combined_cycle_data['Cycle'].min()} ~ {combined_cycle_data['Cycle'].max()}")
    print(f"   • 최대 용량: {combined_cycle_data['DChgCap'].max():.2f} mAh")
    print(f"   • 최소 용량: {combined_cycle_data['DChgCap'].min():.2f} mAh")
    
    # 용량 유지율 계산
    if profile_result.base_capacity > 0:
        final_retention = (combined_cycle_data['DChgCap'].iloc[-1] / profile_result.base_capacity * 100)
        print(f"   • 최종 용량 유지율: {final_retention:.2f}%")
else:
    print(f"   ⚠️  수명 데이터 없음")

print(f"\n💾 데이터베이스:")
print(f"   • 데이터베이스: {db_path}")
print(f"   • 프로젝트: {project.name} (ID: {project.id})")
print(f"   • 테스트 런: {test_run.run_name} (ID: {test_run.id})")

print(f"\n✅ 모든 분석 완료!")
print("="*80)